In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

2025-10-27 10:21:19.328878: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Building Complex Models Using the Functional API
One example of a nonsequential neural network is a Wide & Deep neural network

<img src="https://raw.githubusercontent.com/sambitmukherjee/handson-ml3-pytorch/main/chapter10/Figure_10-13.png" width="480">

In [2]:
housing = fetch_california_housing()
 
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

In [3]:
normalization_layer = tf.keras.layers.Normalization()
hidden_layer1 = tf.keras.layers.Dense(30, activation="relu")
hidden_layer2 = tf.keras.layers.Dense(30, activation="relu")
concat_layer = tf.keras.layers.Concatenate()
output_layer = tf.keras.layers.Dense(1)

In [4]:
input_ = tf.keras.layers.Input(shape=X_train.shape[1:])
normalized = normalization_layer(input_)
hidden1 = hidden_layer1(normalized)
hidden2 = hidden_layer2(hidden1)
concat = concat_layer([normalized, hidden2])
output = output_layer(concat)

2025-10-27 10:21:22.639204: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_COMPAT_NOT_SUPPORTED_ON_DEVICE: forward compatibility was attempted on non supported HW
2025-10-27 10:21:22.639283: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:171] verbose logging is disabled. Rerun with verbose logging (usually --v=1 or --vmodule=cuda_diagnostics=1) to get more diagnostic output from this module
2025-10-27 10:21:22.639299: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:176] retrieving CUDA diagnostic information for host: tumuruu-B85M-D3H
2025-10-27 10:21:22.639306: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:183] hostname: tumuruu-B85M-D3H
2025-10-27 10:21:22.639526: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:190] libcuda reported version is: 580.95.5
2025-10-27 10:21:22.639579: I external/local_xla/xla/stream_

In [5]:
model = tf.keras.Model(inputs=[input_], outputs=[output])

But what if you want to send a subset of the features through the wide path and a different subset (possibly overlapping) through the deep path, In this case, one solution is to use multiple inputs. For example, suppose we want to send five features through the wide path (features 0 to 4), and six features through the deep path (features 2 to 7). We can do this as follows:


<img src="https://raw.githubusercontent.com/sambitmukherjee/handson-ml3-pytorch/main/chapter10/Figure_10-14.png" width="480">

In [6]:
input_wide = tf.keras.layers.Input(shape=[5]) # features 0 to 4
input_deep = tf.keras.layers.Input(shape=[6]) # features 2 to 7

norm_layer_wide = tf.keras.layers.Normalization()
norm_layer_deep = tf.keras.layers.Normalization()

norm_wide = norm_layer_wide(input_wide)
norm_deep = norm_layer_deep(input_deep)

hidden1 = tf.keras.layers.Dense(30, activation="relu")(norm_deep)
hidden2 = tf.keras.layers.Dense(30, activation="relu")(hidden1)

concat = tf.keras.layers.concatenate([norm_wide, hidden2])
output = tf.keras.layers.Dense(1)(concat)

model = tf.keras.Model(inputs=[input_wide, input_deep], outputs=[output])

There are a few things to note in this example, compared to the previous one:

Each Dense layer is created and called on the same line. This is a common practice, as it makes the code more concise without losing clarity. However, we can’t do this with the Normalization layer since we need a reference to the layer to be able to call its adapt() method before fitting the model.



We used tf.keras.layers.concatenate(), which creates a Concatenate layer and calls it with the given inputs.


We specified inputs=[input_wide, input_deep] when creating the model, since there are two inputs

In [7]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss="mse", optimizer=optimizer, metrics=["RootMeanSquaredError"])

In [8]:
X_train_wide, X_train_deep = X_train[:, :5], X_train[:, 2:]
X_valid_wide, X_valid_deep = X_valid[:, :5], X_valid[:, 2:]

X_test_wide, X_test_deep = X_test[:, :5], X_test[:, 2:]
X_new_wide, X_new_deep = X_test_wide[:3], X_test_deep[:3]

In [9]:
norm_layer_wide.adapt(X_train_wide)
norm_layer_deep.adapt(X_train_deep)

history = model.fit((X_train_wide, X_train_deep), y_train, epochs=20, validation_data=((X_valid_wide, X_valid_deep), y_valid))

mse_test = model.evaluate((X_test_wide, X_test_deep), y_test)
y_pred = model.predict((X_new_wide, X_new_deep))

Epoch 1/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - RootMeanSquaredError: 1.2531 - loss: 1.5702 - val_RootMeanSquaredError: 1.2186 - val_loss: 1.4849
Epoch 2/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.7333 - loss: 0.5377 - val_RootMeanSquaredError: 0.6833 - val_loss: 0.4669
Epoch 3/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.6719 - loss: 0.4515 - val_RootMeanSquaredError: 0.6390 - val_loss: 0.4083
Epoch 4/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.6429 - loss: 0.4133 - val_RootMeanSquaredError: 0.6685 - val_loss: 0.4470
Epoch 5/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - RootMeanSquaredError: 0.6272 - loss: 0.3933 - val_RootMeanSquaredError: 0.7591 - val_loss: 0.5762
Epoch 6/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.6189 - loss: 0.3830 - val_RootMeanSquaredError: 1.0333 - val_loss: 1.0678
Epoch 7/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - RootMeanSquaredError: 0.6123 - los

There are also many use cases in which you may want to have multiple outputs:

The task may demand it. For instance, you may want to locate and classify the main object in a picture. This is both a regression tasks and a classification task.

Similarly, you may have multiple independent tasks based on the same data. Sure, you could train one neural network per task, but in many cases you will get better results on all tasks by training a single neural network with one output per task. This is because the neural network can learn features in the data that are useful across tasks. For example, you could perform multitask classification on pictures of faces, using one output to classify the person’s facial expression (smiling, surprised, etc.) and another output to identify whether they are wearing glasses or not.


Another use case is as a regularization technique (i.e., a trainingconstraint whose objective is to reduce overfitting and thus improve the model’s ability to generalize). For example, you may want to add an auxiliary output in a neural network architecture (see Figure 10-15) to ensure that the underlying part of the network learns something useful on its own, without relying on the rest of the network.

<img src="https://raw.githubusercontent.com/sambitmukherjee/handson-ml3-pytorch/main/chapter10/Figure_10-15.png" width="480">

Adding an extra output is quite easy: we just connect it to the appropriate layer and add it to the model’s list of outputs. For example, the following code builds the network represented in Figure 10-15:

In [10]:
output = tf.keras.layers.Dense(1)(concat)
aux_output = tf.keras.layers.Dense(1)(hidden2)

model = tf.keras.Model(inputs=[input_wide, input_deep],outputs=[output, aux_output])

In [11]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

model.compile(loss=("mse", "mse"), loss_weights=(0.9, 0.1), optimizer=optimizer, metrics=[['RootMeanSquaredError'], ['RootMeanSquaredError']])

In [12]:
norm_layer_wide.adapt(X_train_wide)
norm_layer_deep.adapt(X_train_deep)

history = model.fit(
    (X_train_wide, X_train_deep), (y_train, y_train), epochs=20, 
    validation_data=((X_valid_wide, X_valid_deep), (y_valid, y_valid))
)

Epoch 1/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - dense_6_RootMeanSquaredError: 0.9886 - dense_6_loss: 0.9772 - dense_7_RootMeanSquaredError: 1.0119 - dense_7_loss: 1.0237 - loss: 0.9821 - val_dense_6_RootMeanSquaredError: 0.7693 - val_dense_6_loss: 0.5917 - val_dense_7_RootMeanSquaredError: 0.9485 - val_dense_7_loss: 0.8993 - val_loss: 0.6226
Epoch 2/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - dense_6_RootMeanSquaredError: 0.6677 - dense_6_loss: 0.4457 - dense_7_RootMeanSquaredError: 0.7201 - dense_7_loss: 0.5183 - loss: 0.4531 - val_dense_6_RootMeanSquaredError: 0.6269 - val_dense_6_loss: 0.3929 - val_dense_7_RootMeanSquaredError: 0.6968 - val_dense_7_loss: 0.4855 - val_loss: 0.4023
Epoch 3/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - dense_6_RootMeanSquaredError: 0.6292 - dense_6_loss: 0.3959 - dense_7_RootMeanSquaredError: 0.6936 - dense_7_loss: 0.4810 - loss: 0.4044 - val_dense_6_RootMeanSquaredError: 0.5974 - val_dense_6_loss: 0.3568 - val_dense_7_RootMeanSquaredError: 0.6

In [13]:
eval_results = model.evaluate((X_test_wide, X_test_deep), (y_test, y_test))
weighted_sum_of_losses, main_loss, aux_loss, main_rmse, aux_rmse = eval_results

162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - dense_6_RootMeanSquaredError: 0.5697 - dense_6_loss: 0.3244 - dense_7_RootMeanSquaredError: 0.6217 - dense_7_loss: 0.3866 - loss: 0.3308


If you set return_dict=True, then evaluate() will return a dictionary instead of a big tuple.

In [14]:
y_pred_main, y_pred_aux = model.predict((X_new_wide, X_new_deep))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step


In [15]:
y_pred_tuple = model.predict((X_new_wide, X_new_deep))
y_pred = dict(zip(model.output_names, y_pred_tuple))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step


# Using the Subclassing API to Build Dynamic Models